**NOTE** `phimats` environment should be used as kernel

In [63]:
from importlib import reload

import numpy as np

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

import BoundaryConditions
from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

### Simulation data

In [64]:
SimulName = "SENT_EPTH"
# Element sets
nElementSets = 1
# Number of steps to achieve the load
nSteps = 220

### Read mesh file

In [65]:
# Path to .inp mesh file
modelName = "../SENT"

mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 29739
  Number of cells:
    quad: 29390
  Point sets: SENT
  Cell sets: SENT


### Element data

In [66]:
# Element name
elementName = "quad"  		# meshio compatible element name
nNodes = mesh.points.shape[0]            # Total number of nodes
nodCoord = mesh.points[:,0:2]            # Node coordinates
nElements = mesh.cells_dict[elementName].data.shape[0]  # Total number of elements
elemNodeConn = mesh.cells_dict[elementName]        # Node connectivity for all elements

### Mechanical material data

In [ ]:
# Analysis type
AnalysisType = "PlaneStrainPFF"
# Isotropy
Isotropy = "Isotropic"
# Young's modulus
Emod = 210e9
# Poisson's ratio
nu = 0.3

# Plasticity type
Plasticity = "IsoHard"
HardeningLaw = "PowerLaw"
sig_y0 = 500e6    # Pa
K_hard = 885.5e6    # Pa
n_pow = 0.097

# Dislocation evolution (only if you need the normalized dislocation density output, otherwise, set them to zero)

rho_0 = 1e10  # Initial dislocation density (m⁻²)
M = 3 		  # Taylor factor
alpha = 0.3   # Dislocation interaction constant
b = 2.5e-10   # Burgers vector (m)
G = Emod/(2*(1+nu)) # Shear modulus
k1 = 2*(G/(200*G*alpha*b))  # Multiplication coefficient
k2 = 10  # Recovery coefficient

MechMaterial = {
	"Material_1" : {
		"Elastic" : {
			"AnalysisType" : AnalysisType,
			"Isotropy" 	 : Isotropy,
			"Emod" 		 : Emod,
			"nu"   		 : nu,
		},
		"Plastic" : {
			"Plasticity"   : Plasticity,
			"HardeningLaw" : HardeningLaw,
			"sig_y0"       : sig_y0,
			"K_hard"       : K_hard,
			"n_pow"        : n_pow,
   			"rho_0"        : rho_0,
			"M"            : M,
			"b"            : b,
			"alpha"        : alpha,
			"k1"           : k1,
   			"k2"           : k2,
		},
	},
}

### PFF material data

In [ ]:
const_ell = 3e-5     # Length-scale regularization parameter [m]
wc = 11e6            # Critical work density [J/m³]
Gc = 2*const_ell*wc	 # Fracture toughness [J/m²]

PFFMaterial = {
	"Material_1" : {
		"PFF" : {
			"const_ell" : const_ell,
			"wc" 	    : wc,
		}
    }
}

print("l: ", const_ell, " m")
print("Gc: ", Gc, " J/m²")
print("wc: ", wc, " J/m³")

l:  3e-05  m
Gc:  660.0  J/m²
wc:  11000000.0  J/m³


### Apply boundary conditions
**NOTE** This is the total load to be achieved in `nSteps`.

In [69]:
lx = 8.5e-3    # width of the bar
ly = 52e-3     # length of the bar

yDisp = 2.2e-4  # y-disp for top nodes [m]

presBCs = TensileDisp2D(ly, yDisp, mesh)

# Write mesh for inspection 
BCmesh = WriteDispBCs(SimulName, "quad", mesh, presBCs)

### Initialize simulation object

### Mechanical

In [71]:
# Simulation name
Simul = "Mech_"+SimulName
# Simulation type
SimulType = "Mechanical"

SimulationData = {
    "Simul"            : Simul,
	"SimulType"		   : SimulType,
    "mesh"             : mesh,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : presBCs,
    "nSteps"           : nSteps,
    "Materials"        : MechMaterial,
}

MechNotch = PP(SimulationData)
MechNotch.WriteInputFile()

HDF5 file 'Mech_SENT_EPTH_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [72]:
# Create the output file Simul+"_out.hdf5" file
MechNotch.WriteOutputFile(OVERWRITE=True)

### PFF

In [ ]:
# Simulation name
Simul = "PFF_"+SimulName

# Simulation type
SimulType = "PFF"

presBCs = []

SimulationData = {
    "Simul"            : Simul,
	"SimulType"		   : SimulType,
    "mesh"             : mesh,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : presBCs,
    "nSteps"           : nSteps,
    "Materials"        : PFFMaterial,
}

PFFNotch = PP(SimulationData)
PFFNotch.WriteInputFile()

HDF5 file 'PFF_SENT_EPTH_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [75]:
# Create the output file Simul+"_out.hdf5" file
PFFNotch.WriteOutputFile(OVERWRITE=True)

### Write the xdmf file

In [ ]:
WriteXDMF(SimulName, elementName, nSteps+1, "PFF2D")